In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

import pickle 

data_location = "../data/aging_brain/"

young = pickle.load(open(data_location + "aging_brain_young.pickle",mode='rb'))
old = pickle.load(open(data_location + "aging_brain_old.pickle",mode='rb'))

filtered = pickle.load(open(data_location + "aging_brain_filtered.pickle",mode='rb'))

batch_encoding = np.loadtxt(data_location + 'aging_batch_encoding.tsv')
batch_encoding = batch_encoding.astype(dtype=bool)

young_mask = np.zeros(37069,dtype=bool)
old_mask = np.zeros(37069,dtype=bool)

young_mask[:young.shape[0]] = True
old_mask[young.shape[0]:] = True

In [ ]:
# sc.pp.neighbors(filtered)
# sc.tl.umap(filtered)
# sc.tl.louvain(filtered)
# sc.pl.umap(filtered,color='louvain')



In [ ]:
mask_1 = np.zeros(young.X.shape[0],dtype=bool)

mask_1[batch_encoding[young_mask][:,0]] = True
mask_1[batch_encoding[young_mask][:,2]] = True
mask_1[batch_encoding[young_mask][:,4]] = True
mask_1[batch_encoding[young_mask][:,6]] = True

mask_2 = np.zeros(young.X.shape[0],dtype=bool)

mask_2[batch_encoding[young_mask][:,1]] = True
mask_2[batch_encoding[young_mask][:,3]] = True
mask_2[batch_encoding[young_mask][:,5]] = True
mask_2[batch_encoding[young_mask][:,7]] = True

young_1 = young[mask_1]
young_2 = young[mask_2]

In [ ]:
old_mask_1 = np.zeros(old.X.shape[0],dtype=bool)

old_mask_1[batch_encoding[old_mask][:,8]] = True
old_mask_1[batch_encoding[old_mask][:,10]] = True
old_mask_1[batch_encoding[old_mask][:,12]] = True
old_mask_1[batch_encoding[old_mask][:,14]] = True

old_1 = old[old_mask_1]

In [ ]:
import sys
# sys.path.append('/localscratch/bbrener1/rusty_forest_v3/src')
sys.path.append('../src/python/')
import tree_reader as tr 
import lumberjack

data_location = "../data/aging_brain/"

first_forest = tr.Forest.load(data_location + 'scanpy_aging_brain_restricted_1')

In [ ]:
import sys
# sys.path.append('/localscratch/bbrener1/rusty_forest_v3/src')
sys.path.append('../src/python')
import tree_reader as tr 
import lumberjack

first_forest = lumberjack.fit(
    young_1.X,
    header=filtered.var_names,
    trees=100,
    ifs=700,
    ofs=700,
    ss=500,
    depth=8,
    leaves=10,
    dispersion_mode='ssme',
    sfr=0,
    norm='l1',
    reduction=2,
    reduce_input='true',
    reduce_output='false'
)

In [ ]:
first_forest.set_cache(True)
first_forest.backup(data_location + "scanpy_aging_brain_restricted_1")

In [ ]:
first_forest.self_prediction = first_forest.predict(young_1.X)
first_forest.self_prediction.prediction_report()

In [ ]:
feature_residuals = first_forest.self_prediction.residuals()


In [ ]:
centered = young_1.X - np.mean(young_1.X,axis=0)
null_squared_residual = np.power(centered,2)
feature_null = np.sum(null_squared_residual,axis=0) + 1

forest_squared_residuals = np.power(feature_residuals,2)

forest_feature_error = np.sum(forest_squared_residuals,axis=0) + 1
forest_feature_remaining = forest_feature_error/feature_null


In [ ]:
plt.figure()
plt.hist(forest_feature_remaining.copy(),bins=50)
plt.show()

filtered_feature_mask = forest_feature_remaining < .6

print(np.sum(filtered_feature_mask))

In [ ]:
# filtered_feature_mask = np.zeros(2000,dtype=bool)
# for feature in second_forest.output_features:
#     f_i = list(young.var_names).index(feature)
#     filtered_feature_mask[f_i] = True
!pwd

In [ ]:
young_1_filtered = young_1.copy().T[filtered_feature_mask].copy().T
young_2_filtered = young_2.copy().T[filtered_feature_mask].copy().T
old_1_filetered = old_1.copy().T[filtered_feature_mask].copy().T

In [ ]:
second_forest = lumberjack.fit(
    young_1_filtered.X,
    header=young_1_filtered.var_names,
    trees=100,
    ifs=150,
    ofs=150,
    ss=500,
    depth=8,
    leaves=10,
    dispersion_mode='ssme',
    sfr=0,
    norm='l1',
    reduction=2,
    reduce_input='true',
    reduce_output='false'
)

In [ ]:
second_forest.set_cache(True)
second_forest.backup("scanpy_aging_brain_restricted_2")

In [ ]:
second_forest.trim(.01)

In [ ]:
second_forest.self_prediction = second_forest.predict(young_1_filtered.X)
second_forest.self_prediction.node_sample_encoding()

In [ ]:
second_forest.self_prediction.prediction_report()

In [ ]:
second_forest.other_prediction = second_forest.predict(young_2_filtered.X)
second_forest.other_prediction.node_sample_encoding()

In [ ]:
import sys
# sys.path.append('/localscratch/bbrener1/rusty_forest_v3/src')
sys.path.append('../src')
import tree_reader as tr 
import lumberjack
 
second_forest = tr.Forest.load('scanpy_aging_brain_restricted_2')

In [ ]:
second_forest.reset_split_clusters()
second_forest.interpret_splits(
    k=10,
    pca=False, 
    depth=6,
    metric="cosine",  
    mode='additive_mean', 
    relatives=True
)

In [ ]:
second_forest.maximum_spanning_tree(mode='samples')

In [ ]:
second_forest.tsne_coordinates = filtered.obsm['X_umap'][young_mask][mask_1]
second_forest.html_tree_summary(n=10)

In [ ]:
second_forest.split_clusters[20].sisters()

In [ ]:
second_forest.old_prediction = second_forest.predict(old_1_filetered.X)
second_forest.old_prediction.node_sample_encoding()

In [ ]:
self_fvu = []
old_fvu = []
other_fvu = []

other_mwu = []
old_mwu = []

self_fraction = []
other_fraction = []
old_fraction = []

for factor in second_forest.split_clusters[1:]:
    print("++++++++++++++++++++")
    print(factor.name())
    print("++++++++++++++++++++")
#     second_forest.self_prediction.compare_factor_means(second_forest.old_prediction,factor,metric='euclidean',plot=['distance'])
#     second_forest.self_prediction.compare_factor_residuals(second_forest.old_prediction,factor)
    (sfvu,olfvu,olmwu) = second_forest.self_prediction.compare_factor_fvu(second_forest.old_prediction,factor,plot=True)
    (_,otfvu,otmwu) = second_forest.self_prediction.compare_factor_fvu(second_forest.other_prediction,factor,plot=True)
    self_fvu.append(sfvu)
    old_fvu.append(olfvu)
    other_fvu.append(otfvu)
    old_mwu.append(olmwu)
    other_mwu.append(otmwu)
#     (s_f,ot_f,_) = second_forest.self_prediction.compare_factor_fractions(second_forest.old_prediction,factor,plot=True)
#     (s_f,ol_f,_) = second_forest.self_prediction.compare_factor_fractions(second_forest.other_prediction,factor,plot=True)
#     self_fraction.append(s_f)
#     other_fraction.append(ot_f)
#     old_fraction.append(ol_f)
    #     second_forest.self_prediction.compare_factor_fractions(second_forest.other_prediction,factor)
    
#     print(factor.name())
#     sm,om = second_forest.self_prediction.compare_factor_marginals(second_forest.old_prediction,factor)

# print(fraction_return)

self_cod = 1 - np.array(self_fvu) 
old_cod = 1 - np.array(old_fvu)
other_cod = 1 - np.array(other_fvu)

In [ ]:
plt.figure()
plt.title("Coefficient of Determination Ratio")
plt.plot([0,.5],[0,.5],label="Ideal Fit", color='red')
for i,(p1,p2) in enumerate(zip(zip(self_cod,old_cod),zip(self_cod,other_cod))):
    plt.plot([p1[0],p2[0]],[p1[1],p2[1]],linewidth=.5)
    plt.text(p1[0]+.001,p1[1]-.005,str(i+1),fontsize=5)
plt.scatter(self_cod,old_cod,label="Old Mice",s=2)
plt.scatter(self_cod,other_cod,label="Test Set",s=2)
plt.legend()
plt.xlabel("Trained COD")
plt.ylabel("Observed COD")
plt.show()

In [ ]:
# Volcano plot based on MWU results for COD discrepancy. P values seem a bit jumpy, but then again we don't have a
# ton of data points. 

old_lpv = [-np.log10(m[1]) for m in old_mwu]
other_lpv = [-np.log10(m[1]) for m in other_mwu]

plt.figure()
plt.scatter(self_cod - other_cod, other_lpv)
plt.xlabel("Difference in COD")
plt.ylabel("-1og 10 P value")
plt.show()

plt.figure()
plt.scatter(self_cod - old_cod, old_lpv)
for i,(x,y) in enumerate(zip(self_cod-old_cod,old_lpv)):
    plt.text(x+.001,y-.005,str(i+1),fontsize=5)
plt.xlabel("Difference in COD")
plt.ylabel("-1og 10 P value")
plt.show()

plt.figure()
plt.title("Change in COD vs P Value")
plt.scatter(self_cod - other_cod, other_lpv,label='Test',s=2)
plt.scatter(self_cod - old_cod, old_lpv,label='Old',s=2)
plt.xlabel("Difference in COD")
plt.ylabel("-1og 10 P value")
plt.legend()
plt.show()


In [ ]:
plt.figure()
plt.title("Fraction Ratio")
# for i,(p1,p2) in enumerate(zip(zip(self_fraction,old_fraction),zip(self_fraction,other_fraction))):
#     plt.plot([p1[0],p2[0]],[p1[1],p2[1]],linewidth=.5)
#     plt.text(p1[0]+.001,p1[1]-.005,str(i+1),fontsize=5)
plt.scatter(self_fraction,other_fraction,label="Test Set",s=2)
# plt.scatter(self_fraction,old_fraction,label="Old",s=2)
plt.plot([.4,.6],[.4,.6])
plt.legend()
plt.xlabel("Trained Fraction")
plt.ylabel("Observed Fraction")
plt.show()

In [ ]:
cluster = second_forest.split_clusters[5]

cods = []

for node in cluster.nodes:
    (n,p) = second_forest.self_prediction.node_r2_doublet(node)
    fvu = n/p
    cod = 1 - fvu
    cods.append(cod)

plt.figure()
plt.hist(cods,bins=np.arange(-1,1,.01),alpha=.5)

cods = []

for node in cluster.nodes:
    (n,p) = second_forest.old_prediction.node_r2_doublet(node)
    fvu = n/p
    cod = 1 - fvu
    cods.append(cod)

plt.hist(cods,bins=np.arange(-1,1,.01),alpha=.5)
plt.show()

In [ ]:
fraction_return

In [ ]:
for factor in second_forest.split_clusters:
    pops = [len(n.samples()) for n in factor.nodes]
    plt.figure()
    plt.title(factor.name())
    plt.hist(pops,bins=np.arange(0,8000,100))
    plt.show()

In [ ]:
second_forest.self_prediction.prediction_report()

In [ ]:
second_forest.other_prediction.prediction_report()